In [1]:
# 下載檔案
!gdown 1WaKegKk4J3cf1iHghgSRmgrt0MlWIpkq -O ./

Downloading...
From: https://drive.google.com/uc?id=1WaKegKk4J3cf1iHghgSRmgrt0MlWIpkq
To: /content/1Q23-EPR-with-Tables-FINAL.pdf
100% 638k/638k [00:00<00:00, 8.68MB/s]


In [3]:
## 安裝套件
!apt-get install poppler-utils tesseract-ocr -y
!pip install "unstructured pydantic lxml langchain openai chromadb tiktoken

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [4]:
## 請先點選工作列 [執行階段]-> 重新啟動工作階段
## 請先點選工作列 [編輯]-> 清除所有輸出內容
## 右側RAM/磁碟標記 -> 變更執行階段 -> T4 GPU
# Read PDF
from typing import Any
from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf

#1
raw_pdf_elements = partition_pdf(
    filename="./1Q23-EPR-with-Tables-FINAL.pdf",
    extract_images_in_pdf=False,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=".",
)

#2
category_counts = {}

for element in raw_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

unique_categories = set(category_counts.keys())
category_counts

#3
class Element(BaseModel):
    type: str
    text: Any

table_elements = []
text_elements = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        table_elements.append(Element(type="table", text=str(element)))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        text_elements.append(Element(type="text", text=str(element)))

#4
print(len(table_elements))
print(len(text_elements))

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


8
15


In [5]:
# 印出表單
for i, table in enumerate(table_elements):
    print(i, table)

0 type='table' text="($ in millions) ASSETS Current assets: Cash and cash equivalents Merchandise inventory Other current assets Total current assets Property and equipment, net Operating lease assets Other long-term assets Total assets LIABILITIES AND STOCKHOLDERS' EQUITY Current liabilities: Accounts payable Accrued expenses and other current liabilities Current portion of operating lease liabilities Income taxes payable Total current liabilities Long-term liabilities: Revolving credit facility Long-term debt Long-term operating lease liabilities Other long-term liabilities Total long-term liabilities Total stockholders' equity Total liabilities and stockholders' equity April 29, 2023 $ 1,170 2,299 814 4,283 2,646 3,123 880 10,932 $ $ 1,199 1,051 658 10 2,918 350 1,487 3,453 539 5,829 2,185 10,932 $ April 30, 2022 $ 845 3,169 991 5,005 2,791 3,587 874 12,257 $ $ 1,599 1,127 717 29 3,472 350 1,485 3,921 575 6,331 2,454 12,257 $"
1 type='table' text='($ and shares in millions except pe

In [6]:
# 印出段落
for i, text in enumerate(text_elements):
    print(i, text)

0 type='text' text='Gap Inc. Reports First Quarter Fiscal 2023 Results\n\nSAN FRANCISCO – May 25, 2023 – Gap Inc. (NYSE: GPS), a portfolio of purpose-led, billion-dollar lifestyle brands including Old Navy, Gap, Banana Republic, and Athleta, and the largest specialty apparel company in the U.S., today reported financial results for its first quarter ended April 29, 2023.\n\n“We continue to take the necessary actions to drive critical change at Gap Inc., ultimately getting us back on a path toward delivering consistent results long-term,” said Bob Martin, Executive Chairman and Interim CEO, Gap Inc. “While the macro and consumer environment remain uncertain, Q1 underscores our ability to deliver improvements to the business including share gains at Old Navy and Gap Brand, adjusted operating margin expansion, reduction in inventory, and strength in our balance sheet. The need for lasting change is permeating the organization and I want to express my gratitude to our employees for embraci

In [19]:
print('將下列印出的表格一內容貼進去GPT https://chat.openai.com/')
print('')
print('You are responsible for concisely summarizing table or text chunk')
print(table_elements[1].text)

將下列文字貼進去GPT https://chat.openai.com/

You are responsible for concisely summarizing table or text chunk
($ and shares in millions except per share amounts) Net sales Cost of goods sold and occupancy expenses Gross profit Operating expenses Operating loss Interest, net Loss before income taxes Income tax benefit Net loss Weighted-average number of shares - basic Weighted-average number of shares - diluted Net loss per share - basic Net loss per share - diluted 13 Weeks Ended April 29, 2023 April 30, 2022 $ 3,276 2,062 1,214 1,224 (10) 10 (20) (2) (18) $ 3,477 2,381 1,096 1,293 (197) 19 (216) (54) (162) $ $ 367 367 370 370 $ $ (0.05) (0.05) $ $ (0.44) (0.44)
